#import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install plotly_express
!pip install geopandas
!pip install pomegranate

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")
from pomegranate import *
from sklearn import preprocessing
import math

In [4]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_with_wayID'

trips_with_wayId_1_10.csv  trips_with_wayId_1_27.csv  trips_with_wayId_8_15.csv
trips_with_wayId_1_11.csv  trips_with_wayId_1_28.csv  trips_with_wayId_8_16.csv
trips_with_wayId_1_12.csv  trips_with_wayId_1_29.csv  trips_with_wayId_8_17.csv
trips_with_wayId_1_13.csv  trips_with_wayId_1_2.csv   trips_with_wayId_8_18.csv
trips_with_wayId_1_14.csv  trips_with_wayId_1_30.csv  trips_with_wayId_8_19.csv
trips_with_wayId_1_15.csv  trips_with_wayId_1_31.csv  trips_with_wayId_8_20.csv
trips_with_wayId_1_16.csv  trips_with_wayId_1_3.csv   trips_with_wayId_8_21.csv
trips_with_wayId_1_17.csv  trips_with_wayId_1_4.csv   trips_with_wayId_8_22.csv
trips_with_wayId_1_18.csv  trips_with_wayId_1_5.csv   trips_with_wayId_8_23.csv
trips_with_wayId_1_19.csv  trips_with_wayId_1_6.csv   trips_with_wayId_8_24.csv
trips_with_wayId_1_1.csv   trips_with_wayId_1_7.csv   trips_with_wayId_8_25.csv
trips_with_wayId_1_21.csv  trips_with_wayId_1_8.csv   trips_with_wayId_8_26.csv
trips_with_wayId_1_22.csv  trips_with_wa

##get trip data

In [7]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/way_labeled_decrease_NE_2019'
way_id_on_trips_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips'

In [8]:
dfs = []
day_len = []
day_trip_count = []
days = range(1,32)
# days = range(11,12)
months = range(1,2)
# months = range(8,9)

for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(trips_data_path + f'/way_labeled_decrease_2019_{month}_{day}.csv', parse_dates=['time_stamp'])
    except:
      pass

    sp_seq_df = sp_seq_df.sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

# df = df[['trip_id', 'way_id', 'day', 'month','route_cluster']]
df.head()

[23885, 38693, 65653, 93535, 111730, 99358, 84538, 121837, 118054, 113042, 116113, 77262, 107428, 83591, 112616, 92127, 117352, 108331, 117764, 117764, 91649, 78703, 117302, 109436, 100518, 104479, 96250, 68346, 73566, 73575, 74482]
[1036, 1691, 2897, 4190, 4887, 4395, 3837, 5465, 5292, 5121, 5315, 3609, 4782, 3827, 5112, 4273, 5318, 4878, 5288, 5288, 4079, 3668, 5359, 5046, 4598, 4756, 4327, 3197, 3670, 3686, 3641]


,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,way_id,dist,label,month,day
0,2019-01-01 04:26:59,016000200000352253062499073,17.53611,102.75656,53,6.0,1,686957536,0.004565,3092,1,1
1,2019-01-01 04:29:39,016000200000352253062499073,17.54386,102.73553,54,6.0,1,686912257,0.001649,3092,1,1
2,2019-01-01 04:33:39,016000200000352253062499073,17.55599,102.70352,54,6.0,1,686911019,0.002920,3092,1,1
3,2019-01-01 04:45:39,016000200000352253062499073,17.58858,102.60861,51,6.0,1,686910206,0.001777,3092,1,1
4,2019-01-01 05:34:39,016000200000352253062499073,17.69241,102.47342,28,6.0,1,308207967,0.003633,2059,1,1


In [10]:
df_labeled = df

#func score

In [11]:
def score(model, seq, label):
  p = []
  prob, path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  tp = 0
  ###
  # for _state_name, _label in zip(p, label):
  #   if _state_name == _label:
  #     tp+=1
  ###
  # for i, state in enumerate(p):
  #   try:
  #     interval = label[i:i+10]
  #   except:
  #     interval = label[i:]
  #   if state in interval:
  #     tp+=1
  # ### precision
  # total = 0
  # for _state_name, _label in zip(p, label):
  #   if _state_name == '-1':
  #     continue
  #   if _state_name == _label:
  #     tp+=1
  #   total+=1
  ### recal
  # total = 0
  # for _state_name, _label in zip(p, label):
  #   if _state_name == '-1' and _label == '-1':
  #     continue
  #   elif _state_name == _label:
  #     tp+=1
  #   total+=1
  ### precision 10 interval
  total = 0
  for i, state in enumerate(p):
    if state == '-1':
      continue
    try:
      interval = label[i:i+10]
    except:
      interval = label[i:]
    if state in interval:
      tp+=1
    total+=1
  ###
  # total = len(label)
  return tp/total

In [12]:
state_names = []
uniq_label = df_labeled.label.unique()
for label in uniq_label:
  state_names.append(str(label))
state_names[:5]

X = []
labels = []
trip_df = df_labeled
for month in months:
  for day in days:
    day_df = trip_df[(df.day == day) & (trip_df.month == month)]
    uniq_trips = day_df.trip_id.unique()
    for trip_id in uniq_trips:
      trip = day_df[day_df.trip_id==trip_id]
      X.append(trip.way_id.to_numpy().astype(str))
      # labels.append(['None-start']+[str(l) for l in trip.route_cluster.to_list()])
      labels.append([str(l) for l in trip.label.to_list()])
print(X[0][:5])
print(labels[0][:5])
print(len(X))
print(len(labels))

['686957536' '686912257' '686911019' '686910206' '308207967']
['3092', '3092', '3092', '3092', '2059']
132528
132528


In [13]:
import random
random.seed(42)

X2 = X.copy()
labels2 = labels.copy()

temp = list(zip(X2, labels2))
random.shuffle(temp)
X2, labels2 = zip(*temp)
X2 = list(X2)
labels2 = list(labels2)

print(X[0][:5])
print(X2[0][:5])

print(labels[0][:5])
print(labels2[0][:5])

['686957536' '686912257' '686911019' '686910206' '308207967']
['435181731' '91549046' '686978315' '686974410' '686985571']
['3092', '3092', '3092', '3092', '2059']
['1011', '1011', '1011', '2146', '2146']


In [14]:
way_seq = df_labeled.way_id.to_list()
not_stop_mask = [True]+[(a != b) for a, b in zip(way_seq[:-1], way_seq[1:])]
df_labeled = df_labeled.loc[not_stop_mask]

way_df = df_labeled[['way_id', 'lat', 'lon']]
way_df = way_df.drop_duplicates(subset='way_id')

sp_list = set(map(str, way_df['way_id'].tolist()))

In [16]:
total = len(X2)
fold = int(total/10)
start = 0
starts = [] # [0, 952, 1904, 2856, 3808, 4760, 5712, 6664, 7616, 8568]
for i in range(10):
  starts.append(start)
  start+=fold

##for pomegranate

In [ ]:
accuracy_train = []
accuracy_test = []

X_train = X2[:start]+X2[start+fold:]
X_test = X2[start:start+fold]
labels_train = labels2[:start]+labels2[start+fold:]
labels_test = labels2[start:start+fold]

# X_train = X2
# labels_train = labels2
uniq_sp = set()
for seq in X_train:
  for sp in seq:
    uniq_sp.add(sp)
uniq_label = set()

for label in labels_train:
  for uniq in list(set(label)):
    uniq_label.add(uniq)

new_label = []
for label2 in labels_train:
  new_label.append(['None-start']+label2)
labels_train = new_label

unseen_sp = sp_list - uniq_sp
print(len(unseen_sp))
X_train.append(np.array(list(unseen_sp)))
labels_train.append(['-1'] * len(unseen_sp))

model = HiddenMarkovModel.from_samples(
    DiscreteDistribution, 
    n_components=len(uniq_label), 
    X=X_train, 
    labels=labels_train,
    algorithm='labeled',
    state_names=list(uniq_label), 
    inertia=0.001,
    # max_iterations=10,
    max_iterations=1,
    n_jobs=-1
)


0


In [ ]:
# save_model = model.to_json()

In [ ]:
# import json
# with open('/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips/model_1_1.json','w') as f:
#   json.dump(save_model,f)

In [ ]:
# f = open('/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.2_wayid_on_trips/model_1_1.json', "r")
# load_model = json.load(f)
# f.close()
# test = HiddenMarkovModel.from_json(load_model)

In [ ]:
# X_test = X2
# labels_test = labels2

In [ ]:
accuracy_train = []
accuracy_test = []
fail_count = 0
accuracy_train_percent = []
for percent in [0.25, 0.5, 0.75, 0.9]:
  accu = 0
  for seq, label in zip(X_train, labels_train):
    try:
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue
    if sc > 0:
      accu += sc
      # accu += 1
  train_score = accu/len(X_train)
  accuracy_train_percent.append(train_score)
accuracy_train.append(accuracy_train_percent)

accuracy_test_percent = []
for percent in [0.25, 0.5, 0.75, 0.9]:
  accu = 0
  for seq, label in zip(X_test, labels_test):
    try:
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
    except:
      print("Fail!!", end="\t")
      fail_count = fail_count + 1
      continue
    
    if sc > 0:
      accu += sc
      # accu += 1
  test_score = accu/len(X_test)
  accuracy_test_percent.append(test_score)
accuracy_test.append(accuracy_test_percent)


print(f'Train score')
print(f'\t25% trip traverse : {accuracy_train_percent[0]}')
print(f'\t50% trip traverse : {accuracy_train_percent[1]}')
print(f'\t75% trip traverse : {accuracy_train_percent[2]}')
print(f'\t90% trip traverse : {accuracy_train_percent[3]}')
print(f'Test score')
print(f'\t25% trip traverse : {accuracy_test_percent[0]}')
print(f'\t50% trip traverse : {accuracy_test_percent[1]}')
print(f'\t75% trip traverse : {accuracy_test_percent[2]}')
print(f'\t90% trip traverse : {accuracy_test_percent[3]}')
print('fail seq = ',fail_count)

Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!	Fail!!